In [1]:
import pandas as pd
import numpy as np
import json
import random
import nltk
from nltk.corpus import stopwords
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.text import text_to_word_sequence
from keras.layers import Dropout
from tensorflow import keras
from keras import preprocessing
from keras import Model

# Uploading and preprocessing data

In [2]:
#lowercase and removing stopwords
def get_data(filename):
    with open(filename, 'r', encoding='unicode_escape') as f:
        data = f.read()
    data = data.lower()

    nltk.download('stopwords')
    stop_words = stopwords.words('english')
    data = data.split()
    data = [w for w in data if w not in stop_words]
    data = " ".join(data)

    return(data)

In [3]:
it_data = get_data('data/it_oriented.TXT')
non_it_data = get_data('data/ordinary.TXT')
all_data = it_data + non_it_data

all_data[:20]

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


'reason java still re'

In [4]:
#creating word index
def create_index(texts, filename):
    words = texts.split() 
    
    #there can be up to 50 000 words understood by our neural network
    tokenizer = Tokenizer(num_words=50000) 
    
    #we wil be tokenizing all of those words from the text files
    tokenizer.fit_on_texts(words) 
    sequences = tokenizer.texts_to_sequences(words)
    word_index = tokenizer.word_index

    print(f"Found {len(word_index)} unique words") 
    
    with open (filename, 'w') as f:
        json.dump(word_index, f, indent=4)

In [5]:
create_index(all_data, 'word_index.json')

Found 12121 unique words


In [6]:
#grab and return word index
def get_index(filename): #for opening that json file
    with open(filename, 'r') as f:
        data = json.load(f)
    return(data)

In [7]:
word_index = get_index('word_index.json')

In [8]:
#creating sentences
def create_sents(text):
    nltk.download('punkt')
    sentences = nltk.tokenize.sent_tokenize(text)
    return(sentences)

In [9]:
it_sents = create_sents(it_data)
non_it_sents = create_sents(non_it_data)

it_sents[0]

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


'reason java still relevant shoved throats high schoolers college students.'

In [10]:
#padding
def padding_data(sentences, index, maxlen=25):
    new_sentences = []
    for sentence in sentences:
        #this will give us a sentence converted to numerical array
        sentence = text_to_word_sequence(sentence) 
        new_sentence = []
        words = []
        for word in sentence:
            try:
                word = index[word]
            except:
                KeyError
                #for unknown words we encounter:
                word = 0 
            words.append(word)
        new_sentence.append(words)
        new_sentence = preprocessing.sequence.pad_sequences(new_sentence, maxlen=maxlen, padding='post')
        new_sentences.append(new_sentence[0])
    return(new_sentences)

In [11]:
it_padded = padding_data(it_sents, word_index, maxlen=25)
non_it_padded = padding_data(non_it_sents, word_index, maxlen=25)

it_padded[0]

array([ 359,  125,   79, 1165, 6041, 6042,  131, 6043,  849,  278,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0], dtype=int32)

In [12]:
#reverse word index
def reverse_index(word_index):
    reverse_word_index = {value: key for (key, value) in word_index.items()}
    return(reverse_word_index)
    
def reconst_text(text, reverse_word_index):
    return(" ".join([reverse_word_index.get(i, "?") for i in text]))

In [13]:
reverse_word_index = reverse_index(word_index)

reconst_text(it_padded[0], reverse_word_index)

'reason java still relevant shoved throats high schoolers college students ? ? ? ? ? ? ? ? ? ? ? ? ? ? ?'

# Labelling Data

In [14]:
def label_data(sentences, label):
    total_chunks = []
    for sentence in sentences:
        #in form of a tuple that consists of a list of numbers and a label
        total_chunks.append((sentence, label)) 
    return(total_chunks)

In [15]:
non_it_labelled = label_data(non_it_padded, 0)
it_labelled = label_data(it_padded, 1)

non_it_labelled[0]

(array([2779, 1803, 8427, 8428,  464, 5199, 1641,  649, 5200, 5201,  393,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0], dtype=int32),
 0)

# Creating Training Data

In [16]:
def create_training(total_chunks, cutoff):
    random.shuffle(total_chunks)
    training_data = []
    training_labels = []
    testing_data = []
    testing_labels = []
    test_num = len(total_chunks) * cutoff
    x = 0
    
    for entry in total_chunks:
        if x > test_num:
            testing_data.append(entry[0])
            testing_labels.append(entry[1])
        else:
            training_data.append(entry[0])
            training_labels.append(entry[1])
        x = x + 1

    #we are converting all that into a numpy array
    training_data = np.array(training_data) 
    training_labels = np.array(training_labels)
    testing_data = np.array(testing_data)
    testing_labels = np.array(testing_labels)
    
    return(training_data, training_labels, testing_data, testing_labels)

In [17]:
all_data = it_labelled + non_it_labelled
tt_data = create_training(all_data, cutoff = 0.8)

In [18]:
tt_data

(array([[ 5090,  3922,  1663, ...,     0,     0,     0],
        [    9,    55,  1601, ...,     0,     0,     0],
        [ 2722,  2977,   448, ..., 11564,    63,   322],
        ...,
        [ 7614,   805,   377, ...,     0,     0,     0],
        [  494,    16,   111, ...,     0,     0,     0],
        [  346,  4942,  1401, ...,     0,     0,     0]], dtype=int32),
 array([1, 1, 0, ..., 1, 0, 1]),
 array([[  397,    68,     1, ...,     0,     0,     0],
        [   58, 10453,   890, ...,     0,     0,     0],
        [12112,   857,  1830, ...,     0,     0,     0],
        ...,
        [ 8429,  8430,  8431, ...,     0,     0,     0],
        [ 1796,  2836,  3087, ...,     0,     0,     0],
        [ 1451,     0,     0, ...,     0,     0,     0]], dtype=int32),
 array([1, 0, 0, ..., 0, 1, 0]))

In [19]:
#create model
def create_model():
    model = keras.Sequential()
    model.add(keras.layers.Embedding(20000, 25))
    model.add(keras.layers.GlobalAveragePooling1D())
    model.add(keras.layers.Dense(32, activation='relu'))
    model.add(Dropout(0.1))
    model.add(keras.layers.Dense(32, activation='relu'))
    model.add(Dropout(0.1))
    model.add(keras.layers.Dense(16, activation='tanh'))
    model.add(Dropout(0.1))
    model.add(keras.layers.Dense(1, activation='sigmoid'))
    model.summary()
    model.compile(optimizer='adam', loss='squared_hinge', metrics=['accuracy'])
    return(model)

#calling the function
model = create_model()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, None, 25)          500000    
                                                                 
 global_average_pooling1d (G  (None, 25)               0         
 lobalAveragePooling1D)                                          
                                                                 
 dense (Dense)               (None, 32)                832       
                                                                 
 dropout (Dropout)           (None, 32)                0         
                                                                 
 dense_1 (Dense)             (None, 32)                1056      
                                                                 
 dropout_1 (Dropout)         (None, 32)                0         
                                                        

In [20]:
#train model
def train_model(model, tt_data, val_size=.1, epochs=1, batch_size=32):
    vals = int(len(tt_data[0])*val_size)
    training_data = tt_data[0]
    training_labels = tt_data[1]
    testing_data = tt_data[2]
    testing_labels = tt_data[3]
    
    x_val = training_data[:vals]
    x_train = training_data[vals:]
    
    y_val = training_labels[:vals]
    y_train = training_labels[vals:]
    
    fitModel = model.fit(x_train, y_train, epochs=epochs, batch_size=batch_size, validation_data=(x_val, y_val), verbose=1, shuffle=True)
    model_results = model.evaluate(testing_data, testing_labels)
    return model
    
#calling the function
model = train_model(model, tt_data=tt_data, epochs=5, batch_size=32)
model.save('models/it_or_not.model')
model = keras.models.load_model('models/it_or_not.model')


Epoch 1/5
160/160 [==============================] - 6s 23ms/step - loss: 1.0399 - accuracy: 0.5137 - val_loss: 0.9486 - val_accuracy: 0.5176
Epoch 2/5
160/160 [==============================] - 3s 22ms/step - loss: 0.7345 - accuracy: 0.7893 - val_loss: 0.7047 - val_accuracy: 0.8873
Epoch 3/5
160/160 [==============================] - 3s 19ms/step - loss: 0.5949 - accuracy: 0.9260 - val_loss: 0.6717 - val_accuracy: 0.8539
Epoch 4/5
160/160 [==============================] - 3s 17ms/step - loss: 0.5692 - accuracy: 0.9498 - val_loss: 0.6712 - val_accuracy: 0.8820
Epoch 5/5
45/45 [==============================] - 0s 5ms/step - loss: 0.6674 - accuracy: 0.8811
INFO:tensorflow:Assets written to: models/it_or_not.model/assets


# Test model

In [21]:
def prepare(sent):
  prep_sent = [w.lower for w in sent if w not in stopwords.words('english')] #opening, removing stopwords and making lowercase
  prep_sent = str(prep_sent)
  test_sent = create_sents(prep_sent) #turning into num arrays
  test_padded = padding_data(test_sent, word_index, maxlen=25)
  return test_padded

t_sent = "Machine learning (ML) is the study of computer algorithms that can improve automatically through experience and by the use of data."
prepared_sent = prepare(t_sent)

prediction = model.predict(prepared_sent)
print(prediction)


 

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


ValueError: in user code:

    File "/usr/local/lib/python3.8/dist-packages/keras/engine/training.py", line 1621, in predict_function  *
        return step_function(self, iterator)
    File "/usr/local/lib/python3.8/dist-packages/keras/engine/training.py", line 1611, in step_function  **
        outputs = model.distribute_strategy.run(run_step, args=(data,))
    File "/usr/local/lib/python3.8/dist-packages/keras/engine/training.py", line 1604, in run_step  **
        outputs = model.predict_step(data)
    File "/usr/local/lib/python3.8/dist-packages/keras/engine/training.py", line 1572, in predict_step
        return self(x, training=False)
    File "/usr/local/lib/python3.8/dist-packages/keras/utils/traceback_utils.py", line 67, in error_handler
        raise e.with_traceback(filtered_tb) from None
    File "/usr/local/lib/python3.8/dist-packages/keras/engine/input_spec.py", line 213, in assert_input_compatibility
        raise ValueError(f'Input {input_index} of layer "{layer_name}"